In [59]:
import numpy as np
from astropy.table import Table,join
import matplotlib.pyplot as plt

In [60]:
dr3_rv_gaia_bstep = Table.read('VAC_Dynamics_V2_input_topcat.fits')

In [61]:
dr3_rv_gaia_bstep.keys()

['star_id',
 'sobject_id',
 'dr2_source_id',
 'dr3_source_id',
 'rv_galah',
 'e_rv_galah',
 'use_rv_flag',
 'rv_sme_v2',
 'e_rv_sme',
 'rv_obst',
 'e_rv_obst',
 'dr2_radial_velocity',
 'dr2_radial_velocity_error',
 'ra',
 'ra_error',
 'dec',
 'dec_error',
 'parallax_error',
 'pmra',
 'pmra_error',
 'pmdec',
 'pmdec_error',
 'ra_dec_corr',
 'ra_parallax_corr',
 'ra_pmra_corr',
 'ra_pmdec_corr',
 'dec_parallax_corr',
 'dec_pmra_corr',
 'dec_pmdec_corr',
 'parallax_pmra_corr',
 'parallax_pmdec_corr',
 'pmra_pmdec_corr',
 'parallax_corr',
 'r_med_geo',
 'r_lo_geo',
 'r_hi_geo',
 'r_med_photogeo',
 'r_lo_photogeo',
 'r_hi_photogeo',
 'distance_bstep',
 'e_distance_bstep',
 'e16_distance_bstep',
 'e84_distance_bstep']

In [62]:
# Flag for used distance:
# 0: BSTEP (including distance without parallaxes)
# 1: Bailer-Jones photogeo distance
# 2: Bailer-Jones geo distance
# 4: 1/parallax
# 8: None

# initialise assuming bstep is available
use_dist_flag = -1*np.ones(len(dr3_rv_gaia_bstep['sobject_id']),dtype=np.int)

# distance_bstep available
bstep_available = (
    np.isfinite(dr3_rv_gaia_bstep['distance_bstep']) &
    (~dr3_rv_gaia_bstep['distance_bstep'].mask)
)

use_dist_flag[bstep_available] = 0

# go to bailer-jones
bailer_jones_photogeo = (
    (use_dist_flag < 0) &
    np.isfinite(dr3_rv_gaia_bstep['r_med_photogeo']) &
    (~dr3_rv_gaia_bstep['r_med_photogeo'].mask)
)
use_dist_flag[bailer_jones_photogeo] = 1

# go to bailer-jones
bailer_jones_geo = (
    (use_dist_flag < 0) &
    np.isfinite(dr3_rv_gaia_bstep['r_med_geo'])
)
use_dist_flag[bailer_jones_geo] = 2

# go to parallax
one_over_parallax = (
    (use_dist_flag < 0) &
    np.isfinite(dr3_rv_gaia_bstep['parallax_corr'])
)
use_dist_flag[one_over_parallax] = 4

# nothing
no_parallax = (
    np.isnan(dr3_rv_gaia_bstep['distance_bstep']) &
    np.isnan(dr3_rv_gaia_bstep['r_med_photogeo']) &
    np.isnan(dr3_rv_gaia_bstep['r_med_geo']) &
    np.isnan(dr3_rv_gaia_bstep['parallax_corr'])
)
use_dist_flag[no_parallax] = 8

In [63]:
# Flag for used RV
# 0: GALAH RV VAC Zwitter rv_obst (including grav. redshift correction)
# 1: GALAH DR3
# 2: Gaia DR2
# 4: no RV

# initialise assuming bstep is available
use_rv_flag = -1*np.ones(len(dr3_rv_gaia_bstep['sobject_id']),dtype=np.int)

# has zwitter rv_obst
has_zwitter = (
    np.isfinite(dr3_rv_gaia_bstep['rv_obst']) &
    np.isfinite(dr3_rv_gaia_bstep['e_rv_obst'])
)
use_rv_flag[has_zwitter] = 0

# no zwitter rv, so go to rv_galah
no_zwitter = (
    (use_rv_flag < 0) &
    (
        np.isfinite(dr3_rv_gaia_bstep['rv_sme_v2']) &
        np.isfinite(dr3_rv_gaia_bstep['e_rv_sme'])
    )
)
use_rv_flag[no_zwitter] = 1

# go to gaia dr2 rv, if GALAH RV not available or GALAH RV error larger than Gaia DR2
use_gaia_rv = (
    (
        (use_rv_flag < 0) &
        np.isfinite(dr3_rv_gaia_bstep['dr2_radial_velocity']) &
        np.isfinite(dr3_rv_gaia_bstep['dr2_radial_velocity_error'])        
    ) |
    (
        (use_rv_flag == 0) &
        np.isfinite(dr3_rv_gaia_bstep['dr2_radial_velocity']) &
        np.isfinite(dr3_rv_gaia_bstep['dr2_radial_velocity_error']) &   
        (dr3_rv_gaia_bstep['dr2_radial_velocity_error'] < dr3_rv_gaia_bstep['e_rv_obst'])
    ) |
    (
        (use_rv_flag == 1) &
        np.isfinite(dr3_rv_gaia_bstep['dr2_radial_velocity']) &
        np.isfinite(dr3_rv_gaia_bstep['dr2_radial_velocity_error']) &   
        (dr3_rv_gaia_bstep['dr2_radial_velocity_error'] < dr3_rv_gaia_bstep['e_rv_sme'])
    )   
)
use_rv_flag[use_gaia_rv] = 2

# no rv
no_rv = (
    use_rv_flag < 0
)
use_rv_flag[no_rv] = 4

In [64]:
np.where(dr3_rv_gaia_bstep['use_rv_flag'] - use_rv_flag != 0)

(array([  1677,   1680,   1681, ..., 678380, 678383, 678391]),)

In [65]:
dr3_rv_gaia_bstep['use_dist_flag'] = use_dist_flag
dr3_rv_gaia_bstep['use_rv_flag'] = use_rv_flag
dr3_rv_gaia_bstep.write('VAC_Dynamics_V2_input.fits',overwrite=True)

In [66]:
for each in [-1,0,1,2,4,8]:
    print(len(np.where(use_dist_flag==each)[0]))

0
649453
27639
41
0
1290


In [67]:
for each in [-1,0,1,2,4]:
    print(len(np.where(use_rv_flag==each)[0]))

0
563260
86173
14190
14800
